The backtracking method can be generalized to construct a longest path in any DAG.
Whenever we compute sb as

sb = max all predecessors a of node b {sa + weight of edge from a to b},

we simply need to store a predecessor of b that was used in the computation of sb so that
we can backtrack later on.

It can be proven that any DAG has a topological ordering, and that this topological ordering can be constructed in time proportional to the number of edges in the graph. Once we have a topological ordering, we can compute the length of the longest path from source to sink by visiting the nodes of the DAG in the order dictated by the topological ordering, which is achieved by the following algorithm. For simplicity, we assume that the source node is the only node with indegree 0 in Graph. --> pretpostavka, može biti više čvorova sa indegree 0 --> graf nije čvrsto povezan ako ih ima više --> za source node uzmemo zadani source node 

LONGESTPATH(Graph, source, sink)

for each node b in Graph

sb = minus infinity

ssource 0

topologically order Graph

for each node b in Graph (following the topological order)

sb maxall predecessors a of node b {sa + weight of edge from a to b}

return ssink


In [20]:
def FormatGraph(graph):
  edges_dict = {}
  for i in range(len(graph)):
    graph[i] = graph[i].split('->')
    graph[i][1] =  graph[i][1].split(':')
    edges_dict.setdefault((int(graph[i][0]),int(graph[i][1][0])),[])
    edges_dict[(int(graph[i][0]),int(graph[i][1][0]))].append(int(graph[i][1][1]))
    graph[i] = [int(graph[i][0])] + [int(graph[i][1][0])]
  return [graph,edges_dict]

In [21]:
def GraphNodes(formatted_graph):
  graph_nodes = set()
  for nodes in formatted_graph:
    graph_nodes.update(nodes)
  return list(graph_nodes)

In [22]:
def GraphDict(formatted_graph):
  graph_dict = {}
  graph_nodes = GraphNodes(formatted_graph)
  for graph_node in graph_nodes:
    graph_dict.setdefault(graph_node,[])
  for i in range(len(formatted_graph)):
    adjacent_nodes = formatted_graph[i][1:len(formatted_graph[i])]
    for adjacent_node in adjacent_nodes:
      graph_dict[formatted_graph[i][0]].append(adjacent_node)
  return graph_dict

In [23]:
def NodeInDegree(graph_dict,node):
  node_in_degree = 0
  for adjacent_nodes in graph_dict.values():
    node_in_degree = node_in_degree + adjacent_nodes.count(node)
  return node_in_degree

In [24]:
def NonVisitedEdgesNumber(graph_dict):
  non_visited_edges_number = 0
  for adjacent_nodes in graph_dict.values():
    non_visited_edges_number = non_visited_edges_number + len(adjacent_nodes)
  return non_visited_edges_number

In [25]:
def TopologicalOrdering(graph_dict):
  topological_ordering = list()
  candidates = [node for node in graph_dict.keys() if NodeInDegree(graph_dict,node) == 0]
  while len(candidates) > 0:
    a = candidates[0]
    topological_ordering.append(a)
    candidates.pop(0)
    adjacent_nodes = graph_dict[a].copy()
    graph_dict[a] = [] #remove all outgoing edges
    candidates.extend([adjacent_node for adjacent_node in adjacent_nodes if NodeInDegree(graph_dict,adjacent_node) == 0])
  if NonVisitedEdgesNumber(graph_dict) > 0: #if graph has edges that have not been removed
    return "the input graph is not a DAG"
  else:
    return topological_ordering

In [26]:
import numpy as np

In [27]:
def FindPredcessors(graph_dict,node):
  predcessors = []
  for key in graph_dict.keys():
    if graph_dict[key].count(node) > 0:
      predcessors.append(key)
  return predcessors

The following algorithm for constructing a topological ordering is based on the observation that every DAG has at least one node with no incoming edges (zato što je directed **acylic** graph, prvi čvor u topološkom redoslijedu nema ulaznih čvorova pa je on izvorni čvor, njega moramo prvoga posjetiti jer je on predcessor nekim čvorovima). We will label
one of these nodes as v1 and then remove this node from the graph along with all its outgoing edges. The resulting graph is also a DAG, which in turn must have a node with no incoming edges; we label this node v2, and again remove it from the graph along with its outgoing edges. The resulting algorithm proceeds until all nodes have been removed, producing a topological order v1, ... , vn

In [336]:
def DirectedAcylicGraphLongestPath(source_node,sink_node,graph):
  formatted_graph,edges_dict = FormatGraph(graph)
  graph_dict = GraphDict(formatted_graph)
  topological_ordering = TopologicalOrdering(graph_dict.copy())
  s = {}
  backtrack = {}
  if topological_ordering.index(source_node) > 0:
    for node in topological_ordering[0:topological_ordering.index(source_node)+1]:
      s.update({node:0}) #key-->node, value-->length of longest path to node
      backtrack.update({node:str()})
  else:
    #if source node is not the first node in topological ordering then we set topological ordering to start from the source node --> all previous nodes have s = 0 as they cannot be reached from any other nodes
    #last node of topological ordering does not need to be sink node as there can be multiple nodes with no out edges
    topological_ordering = topological_ordering[topological_ordering.index(source_node):topological_ordering.index(sink_node)+1] 
    for node in topological_ordering:
      s.update({node:np.NINF}) #key-->node, value-->length of longest path to node
      backtrack.update({node:str()})
  s[source_node] = 0
  for i in range(1,len(topological_ordering)): #we don't need to visit the first node as it has no incoming edges and s[first_node] = 0, last node is the sink node
      terms= []
      predcessors = FindPredcessors(graph_dict,topological_ordering[i])
      if len(predcessors) > 0:
        for predcessor in predcessors:
            for edge_weight in edges_dict[(predcessor,topological_ordering[i])]: #predcessor --> node that has outgoing edges to the topological_ordering[i]
                terms.append(s[predcessor] + edge_weight)
        s[topological_ordering[i]] = max(terms) #based on recurrence relation, ties are broken arbitrarily
        for predcessor in predcessors:
            for edge_weight in edges_dict[(predcessor,topological_ordering[i])]: #predcessor can have multiple outgoing nodes to topological_ordering[i]
              if s[topological_ordering[i]] == s[predcessor] + edge_weight:
                backtrack[topological_ordering[i]] = str(predcessor)
      else:
        s[topological_ordering[i]] = 0
  return edges_dict,backtrack

In [305]:
def OutputLongestPath(edges_dict,backtrack,source_node,sink_node):
  longest_path_length = 0
  current_node = sink_node
  longest_path = [current_node]
  while backtrack[current_node] != '': #while we don't reach source node --> backtrack[source_node] = ''
    longest_path_length = longest_path_length + edges_dict[(int(backtrack[current_node]),current_node)][0]
    current_node = int(backtrack[current_node])
    longest_path.append(current_node)
  return longest_path_length,longest_path[::-1]

In [306]:
def PrintResultToFile(longest_path_length,longest_path):
  f = open("task_result.txt","w")
  f.write(str(longest_path_length) + '\n')
  string_to_print = ''
  for node in longest_path:
    string_to_print = string_to_print + str(node) + '->'
  f.write(string_to_print[0:len(string_to_print)-2])
  f.close()

In [307]:
source_node = 0

In [308]:
sink_node = 4

In [309]:
graph = [
'0->1:7',
'0->2:4',
'2->3:2',
'1->4:1',
'3->4:3']

In [310]:
edges_dict,backtrack = DirectedAcylicGraphLongestPath(source_node,sink_node,graph)

In [311]:
longest_path_length,longest_path = OutputLongestPath(edges_dict,backtrack,source_node,sink_node)

In [312]:
PrintResultToFile(longest_path_length,longest_path)

In [337]:
with open('/content/rosalind_ba5d.txt') as task_file:
  task_arguments = [line.rstrip() for line in task_file]

In [338]:
source_node = int(task_arguments[0])

In [339]:
sink_node = int(task_arguments[1])

In [340]:
graph = task_arguments[2:len(task_arguments)]

In [341]:
edges_dict,backtrack = DirectedAcylicGraphLongestPath(source_node,sink_node,graph)

In [342]:
longest_path_length,longest_path = OutputLongestPath(edges_dict,backtrack,source_node,sink_node)

In [343]:
PrintResultToFile(longest_path_length,longest_path)